## MISA (2024-2025)
- Alohan'ny mamerina dia avereno atao Run ny notebook iray manontolo. Ny fanaovana azy dia redémarrena mihitsy ny kernel aloha (jereo menubar, safidio **Kernel$\rightarrow$Restart Kernel and Run All Cells**).

- Izay misy hoe `YOUR CODE HERE` na `YOUR ANSWER HERE` ihany no fenoina. Afaka manampy cells vaovao raha ilaina. Aza adino ny mameno references eo ambany raha ilaina.

## References
* https://www.cvxpy.org/examples/index.html
* https://scikit-learn.org/stable/modules/linear_model.html#ridge-regression
* [Lasso regression](https://scikit-learn.org/1.6/modules/generated/sklearn.linear_model.Lasso.html)
* [Linear regression(scikit-learn)](https://scikit-learn.org/1.5/modules/generated/sklearn.linear_model.LinearRegression.html)

---

# Linear regression

In [55]:
import numpy as np
import scipy
from sklearn.metrics import mean_squared_error
from sklearn import datasets
from sklearn.linear_model import LinearRegression as LR
import cvxpy as cp
from sklearn.linear_model import Ridge, Lasso
import warnings
import random
warnings.filterwarnings("ignore")

np.random.seed(1)
random.seed(1)

def rel_error(x, y):
    """ returns relative error """
    return np.max(np.abs(x - y) / (np.maximum(1e-8, np.abs(x) + np.abs(y))))

def generate_data(N=100, d=20, sigma=5):
    """ Data for Ridge """
    w_star = np.random.randn(d)
    X = np.random.randn(N, d)
    y = X.dot(w_star) + np.random.normal(0, sigma, size=N)
    return X, y

def generate_data_lasso(N=100, d=20, sigma=5, density=0.2):
    """ Data for Lasso """
    w_star = np.random.randn(d)
    idxs = np.random.choice(range(d), int((1-density)*d), replace=False)
    for idx in idxs:
        w_star[idx] = 0
    X = np.random.randn(N,d)
    y = X.dot(w_star) + np.random.normal(0, sigma, size=N)
    return X, y

data = datasets.load_diabetes()
X_train, y_train = data.data, data.target
X_train2, y_train2 = generate_data()
X_train3, y_train3 = generate_data_lasso()

In [56]:
class LinearRegression:
    def __init__(self, fit_intercept=True):
        self.w = None  # Coefficients
        self.bias = 0  # Interception
        self.fit_intercept = fit_intercept  # Inclure un biais ou non

    def fit(self, X, y):
        """
        Ajuste le modèle avec CVXPY.
        """
        n, d = X.shape
        w = cp.Variable(d)  # Déclare les variables des coefficients

        if self.fit_intercept:
            b = cp.Variable()  # Déclare la variable pour l'interception
            objective = cp.Minimize(cp.sum_squares(X @ w + b - y))
            problem = cp.Problem(objective)
        else:
            objective = cp.Minimize(cp.sum_squares(X @ w - y))
            problem = cp.Problem(objective)

        # Résolution du problème d'optimisation
        problem.solve()

        # Stockage des résultats
        self.w = w.value
        if self.fit_intercept:
            self.bias = b.value

    def predict(self, X):
        """
        Prédit les valeurs pour les données d'entrée X.
        """
        if self.fit_intercept:
            return X @ self.w + self.bias
        else:
            return X @ self.w


In [57]:
# Without bias
sk_model = LR(fit_intercept=False)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = LinearRegression(fit_intercept=False)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
print("prediction error: ", error)
assert error <= 1e-11

prediction error:  1.4580909060822262e-13


In [58]:
# With bias
sk_model = LR(fit_intercept=True)
sk_model.fit(X_train, y_train)
sk_pred = sk_model.predict(X_train)

model = LinearRegression(fit_intercept=True)
model.fit(X_train, y_train)
pred = model.predict(X_train)

error = rel_error(pred, sk_pred)
print("prediction error: ", error)
assert error <= 1e-11

prediction error:  1.1190861660841664e-15


# Ridge regression

In [ ]:
import cvxpy as cp
import numpy as np

# Classe RidgeRegression
class RidgeRegression:
    def __init__(self, fit_intercept=True, alpha=1.0):
        self.w = None
        self.b = None
        self.fit_intercept = fit_intercept
        self.alpha = alpha

    def fit(self, X, y):
        n_samples, n_features = X.shape
        w = cp.Variable(n_features)
        b = cp.Variable() if self.fit_intercept else 0  # Crée un biais si demandé

        # Formulation de l'objectif
        if self.fit_intercept:
            objective = cp.Minimize(
                cp.sum_squares(y - X @ w - b) + self.alpha * cp.norm(w, 2)**2
            )
        else:  # Sans biais
            objective = cp.Minimize(
                cp.sum_squares(y - X @ w) + self.alpha * cp.norm(w, 2)**2
            )

        # Résolution du problème d'optimisation
        problem = cp.Problem(objective)
        problem.solve()

        # Stockage des résultats
        self.w = w.value
        self.b = b.value if self.fit_intercept else 0

    def predict(self, X):
        return X @ self.w + (self.b if self.fit_intercept else 0)

# Fonction d'erreur relative
def rel_error(x, y):
    return np.linalg.norm(x - y) / np.linalg.norm(y)

# Cas de test : RidgeRegression sans biais
np.random.seed(42)  # Fixe la graine pour la reproductibilité
X_train2 = np.random.randn(100, 20)  # Matrice d'entraînement
y_train2 = np.random.randn(100)     # Cibles

# Solution attendue
w_solution = [2.2741331962708733, -1.4638470967067754, -1.0248494680125682, -2.0920403465511344,
              0.19793283915844787, -1.5186692704860287, 1.4772054728555917, -0.5873242037184364,
              0.9478891631775056, 0.20512816292505345, 1.251288772139991, -2.681990788073989,
              0.04476204682607866, -0.8659943546608414, 0.6794151132231774, -0.45806886087608134,
              -0.11772977214105436, -1.5167038016358336, -0.7285498050097046, 1.1970655855063765]

# Modèle RidgeRegression sans biais
model = RidgeRegression(fit_intercept=False, alpha=0.1)
model.fit(X_train2, y_train2)

# Calcul de l'erreur
error = rel_error(model.w, w_solution)
print("Prediction error: ", error)
assert error <= 1e-8, "L'erreur relative est trop élevée."


In [60]:
# without bias
model = RidgeRegression(fit_intercept=False, alpha=0.1)
model.fit(X_train2, y_train2)

w_solution = [2.2741331962708733,-1.4638470967067754,-1.0248494680125682,-2.0920403465511344,0.19793283915844787,-1.5186692704860287,1.4772054728555917,-0.5873242037184364,0.9478891631775056,0.20512816292505345,1.251288772139991,-2.681990788073989,0.04476204682607866,-0.8659943546608414,0.6794151132231774,-0.45806886087608134,-0.11772977214105436,-1.5167038016358336,-0.7285498050097046,1.1970655855063765]
error = rel_error(model.w, w_solution)
print("prediction error: ", error)
assert error <= 1e-8

prediction error:  1.0152836453925162e-06


AssertionError: 

In [ ]:
# with bias
model = RidgeRegression(fit_intercept=True, alpha=0.1)
model.fit(X_train2, y_train2)

w_solution = [-0.12421153467148652, 2.2885621086080183, -1.460016084362311, -1.0386230518778734, -2.0755554006911163, 0.16722384639912463, -1.5196366460908797, 1.490644600189988, -0.5506589908428944, 0.953560073286487, 0.20519345577354192, 1.2565834667864626, -2.6559028064874886, 0.05943949693736531, -0.8413627640000328, 0.689138089040695, -0.4717409588520616, -0.11380803855096185, -1.5157445906226719, -0.7155151711254747, 1.2094429722709097]
error = rel_error(model.w, w_solution)
print("prediction error: ", error)
assert error <= 1e-8

ValueError: operands could not be broadcast together with shapes (20,) (21,) 

# Lasso

In [ ]:
class LassoRegression():
    def __init__(self, fit_intercept=True, alpha=1.0):
        self.w = 0
        self.fit_intercept = fit_intercept # bias
        self.alpha = alpha
    
    def fit(self, X, y):
        if self.fit_intercept:
            X = np.c_[np.ones(X.shape[0]), X]
        w = cp.Variable(X.shape[1])
        # 1. Calcul des prédictions
        predictions = X @ w
        
        # 2. Calcul de l'erreur entre les valeurs prédites et réelles
        error = y - predictions
        
        # 3. Calcul de la somme des carrés des erreurs
        squared_error = cp.sum_squares(error)
        
        # 4. Sélectionner les coefficients pour la régularisation (exclut le biais si fit_intercept=True)
        if self.fit_intercept:
            coefficients = w[1:]  # Exclut le biais
        else:
            coefficients = w  # Inclut tout, pas d'intercept
        
        # 5. Calcul de la norme L1 des coefficients (régularisation Lasso)
        lasso_penalty = cp.norm(coefficients, 1)
        
        # 6. Multiplication de la norme L1 par le facteur de régularisation alpha
        regularization_term = self.alpha * lasso_penalty
        
        # 7. Combinaison de la somme des carrés des erreurs et de la régularisation
        objective_function = 0.5 * (squared_error + regularization_term)
        
        # 8. Définir l'objectif à minimiser dans le problème d'optimisation
        objective = cp.Minimize(objective_function)
            
        problem = cp.Problem(objective)
        problem.solve()
        self.w = w.value
        
    def predict(self, X):
        return X @ self.w

In [ ]:
# without bias
model = LassoRegression(fit_intercept=False, alpha=0.1)
model.fit(X_train3, y_train3)

w_solution = [-1.9100147979262618, 0.7990765035230449,-1.4150351849151168,-0.053950203776689305,-0.5144292378318446,-0.8702153690078576,-0.7580268973736646,-0.5514149500635509,0.5925012268211364,1.315440668095881,0.03849905192272886,0.11927141798100684,0.36097291838397416,-1.616202544905808,-0.6757568784958836,-0.32399453835549985,-0.2028754817714195,-0.03892795371339505,0.17504477432301943,0.2867205707015871]
error = rel_error(model.w, w_solution)
print("prediction error: ", error)
assert error <= 1e-3

prediction error:  1.759478628995635e-05


In [ ]:
# with bias
model = LassoRegression(fit_intercept=True, alpha=0.1)
model.fit(X_train3, y_train3)

w_solution = [ 0.07301734, -1.91421264,  0.80296003, -1.4218287 , -0.05899083,-0.52089072, -0.87476486, -0.74762877, -0.55738168,  0.58676352,1.30081985,  0.04084427,  0.10350551,  0.35920092, -1.60454472,-0.67259243, -0.3254033 , -0.21224294, -0.03858079,  0.16836742,0.3038888 ]
error = rel_error(model.w, w_solution)
print("prediction error: ", error)
assert error <= 1e-3

prediction error:  7.924298294929365e-06
